In [ ]:
import sys
from pathlib import Path
import warnings
import json
import time
import matplotlib.pyplot as plt

# Add parent directory to path to access dot_pe modules
sys.path.insert(0, str(Path.cwd().parent))

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

lal.swig_redirect_standard_output_error(False)

import numpy as np
from cogwheel import gw_utils, utils

# Import modules from the dot_pe package
from dot_pe import sample_generation, config, waveform_creation

In [ ]:
# select parameters for bank
n_intrinsic_samples = 2**10

q_min = 1/6
mchirp_min = 5
mchirp_max = 5.1
bank_name = 'test_bank'
# samples_bank_folder = Path(f'/BIGDATA/jonatahm/evidence_calculator/bank_mchirp_{mchirp_min}_{mchirp_max}')
# samples_bank_folder = Path(f'/BIGDATA/jonatahm/evidence_calculator/{bank_name}')
samples_bank_folder = Path('.')/ 'test_bank'
dataframe_path = samples_bank_folder / 'intrinsic_sample_bank.feather'

isg = sample_generation.IntrinsicSamplesGenerator()
# create folder if it does not exist
if samples_bank_folder.exists() is False:
    samples_bank_folder.mkdir()

In [ ]:
start_time = time.time()
samples = isg.draw_intrinsic_samples_uniform_in_lnmchrip_lnq(
    n_intrinsic_samples, q_min, mchirp_min, mchirp_max,f_ref = 50.0)
samples.to_feather(dataframe_path)

end_time = time.time()
message = (f"{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))} :"
           +f" func finished in {end_time - start_time:.2f} seconds.")
print(message)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].scatter(samples['m1'], samples['m2'], s=1, alpha=0.5)
mchirp =gw_utils.m1m2_to_mchirp(samples['m1'], samples['m2'])
q = np.log(samples['m1']/samples['m2'])
axs[1].scatter(np.log(mchirp), q, s=1, alpha=0.5)

In [ ]:
# create config file

bank_config_path = samples_bank_folder / "bank_config.json"
with open(bank_config_path, "w", encoding="utf-8") as fp:
    config_dict = {"fbin": config.DEFAULT_FBIN, "f_ref": 50.0}
    json.dump(config_dict, fp, cls=utils.NumpyEncoder)

waveform_creation.create_waveform_bank_from_samples(
    samples_path=dataframe_path,
    bank_config_path=bank_config_path,
    waveform_dir=dataframe_path.parent / "waveforms",
    n_blocks=None,
    n_pool=1,
    blocksize=64,
)
